In [1]:
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt

In [5]:
#employee_df = pd.read_csv(r'C:\Users\jakob\Desktop\Projektseminar\Data\employees.csv', delimiter= '|')
#absences_df = pd.read_csv(r'C:\Users\jakob\Desktop\Projektseminar\Data\absences.csv', delimiter='|')
#weather_df = pd.read_csv(r'C:/Users/jakob/Downloads/weather_wuerz.csv', delimiter=',')

employee_df = pd.read_csv("../../data/raw/employees.csv", delimiter= '|')
absences_df = pd.read_csv("../../data/raw/absences.csv", delimiter= '|')

In [6]:
# drop NaN values and delete data for 2024 because to few data
absences_df = absences_df.dropna(subset=['start_date', 'end_date'])
absences_df = absences_df[absences_df['start_date'].str.contains('2024') == False]


Do following only once and load dataframe 'absences_each_missday_a_row' for faster access
Add rows for every booked absence, so each day is one person missing on one day.

In [7]:
# Step 1: Create an empty list to store the rows of the new DataFrame
new_rows = []

# Step 2: Iterate over the original DataFrame row by row
for index, row in absences_df.iterrows():
    # Step 3: Generate a list of dates between 'start_date' and 'end_date' (inclusive)
    date_range = pd.date_range(start=row['start_date'], end=row['end_date'])
    
    # Step 4: Duplicate the row for each date in the list and add a new column with the date value
    for date in date_range:
        new_row = row[['employee_id', 'reason', 'allocation_id']].copy()
        new_row['date'] = date
        new_rows.append(new_row)

# Step 6: Create a new DataFrame using the list of duplicated rows
new_df = pd.DataFrame(new_rows)

# Reset the index of the new DataFrame
new_df.reset_index(drop=True, inplace=True)

# save df for faster reload/access
#new_df.to_csv("../../data/interim/absences_each_missday_a_row.csv", index=False)

# Display the new DataFrame
new_df


,employee_id,reason,allocation_id,date
0,9827,vacation,98044.0,2020-01-17
1,9827,vacation,98044.0,2020-01-18
2,9827,vacation,118649.0,2020-01-17
3,9827,vacation,118649.0,2020-01-18
4,1366,vacation,2679.0,2020-01-20
...,...,...,...,...
301975,9521,vacation,NaN,2022-11-29
301976,9521,vacation,NaN,2022-11-30
301977,9521,vacation,NaN,2022-12-01
301978,9521,vacation,NaN,2022-12-02


In [8]:
# read dataframe for faster access
#my_df = pd.read_csv("../../data/interim/absences_each_missday_a_row.csv", delimiter=',')


In [10]:
# order new_df by date
my_df = new_df.sort_values(by=['date'])
min_date = absences_df['start_date'].min()
max_date = absences_df['start_date'].max()

# Create a DataFrame with the date_range
date_range = pd.date_range(start=min_date, end=max_date)
all_dates = pd.DataFrame({'Date': date_range})
my_df['date'] = pd.to_datetime(my_df['date'])


In [11]:
# join the two DataFrames
new_df = pd.merge(all_dates, my_df, how='left', left_on='Date', right_on='date')
#drop Date and employee_id
new_df = new_df.drop(['date', 'employee_id'], axis=1)
new_df

,Date,reason,allocation_id
0,2020-01-02,vacation,101370.0
1,2020-01-02,vacation,101371.0
2,2020-01-03,vacation,101370.0
3,2020-01-03,vacation,101371.0
4,2020-01-04,vacation,101370.0
...,...,...,...
301499,2023-12-29,vacation,NaN
301500,2023-12-29,vacation,273272.0
301501,2023-12-29,vacation,315562.0
301502,2023-12-29,vacation,311314.0


In [17]:
# change column name to date
new_df = new_df.rename(columns={'Date': 'date'})

aggregated_df = new_df.groupby('date').size().reset_index(name='count')

# Set 'date' column as the index
aggregated_df.set_index('date', inplace=True)

# Generate the complete date range from the minimum to the maximum date in the original dataframe
date_range = pd.date_range(start=new_df['date'].min(), end=new_df['date'].max(), freq='D')

# Reindex the aggregated dataframe to include missing dates and fill the NaN with zero
aggregated_df = aggregated_df.reindex(date_range, fill_value=0).reset_index()

aggregated_df

,index,count
0,2020-01-02,2
1,2020-01-03,2
2,2020-01-04,2
3,2020-01-05,1
4,2020-01-06,1
...,...,...
1453,2023-12-25,60
1454,2023-12-26,60
1455,2023-12-27,213
1456,2023-12-28,211


In [18]:
aggregated_df['day_of_week'] = aggregated_df['index'].dt.dayofweek
aggregated_df['month'] = aggregated_df['index'].dt.month
aggregated_df['season'] = aggregated_df['index'].dt.quarter
# save dataframe for faster access

aggregated_df.to_csv("../../data/interim/absences_per_day.csv", index=False)
aggregated_df

,index,count,day_of_week,month,season
0,2020-01-02,2,3,1,1
1,2020-01-03,2,4,1,1
2,2020-01-04,2,5,1,1
3,2020-01-05,1,6,1,1
4,2020-01-06,1,0,1,1
...,...,...,...,...,...
1453,2023-12-25,60,0,12,4
1454,2023-12-26,60,1,12,4
1455,2023-12-27,213,2,12,4
1456,2023-12-28,211,3,12,4
